In [5]:
import numpy as np

def get_sampled_coords( patch_h, patch_w, H, W):

    sampled_coords = []                 

    sample_x = np.random.choice(W-patch_w, size=[1])
    sample_y = np.random.choice(H-patch_h, size=[1])
    total_num = (W-patch_w) * (H-patch_h) 
    probs = []
    
    for i in range(patch_h):
        for j in range(patch_w):
            sampled_coords.append([i, j])
            total_num = total_num + max(patch_h-i, patch_w-j)
    for i in range(H-2*patch_h, H-patch_h):
        for j in range(W-2*patch_w, W-patch_w):
            sampled_coords.append([i, j])
            total_num = total_num + max(i + 2 * patch_h - H + 1, j + 2 * patch_w - W + 1)


    for i in range(patch_h):
        for j in range(patch_w):
            temp = max(patch_h-i, patch_w-j) / total_num
            probs.append(temp)            
    for i in range(H-2*patch_h, H-patch_h):
        for j in range(W-2*patch_w, W-patch_w):
            temp = max(i + 2 * patch_h - H + 1, j + 2 * patch_w - W + 1) / total_num
            probs.append(temp)
            
    sampled_coords.append([sample_y[0], sample_x[0]])
    temp = (W-patch_w) * (H-patch_h) / total_num
    probs.append(temp)
    
    return sampled_coords, probs

In [8]:
patch_h, patch_w, h, w = 32, 32, 200, 200
coords, probs = get_sampled_coords(patch_h, patch_w, h, w)

print(len(coords))
print(len(probs))

2049
2049


In [ ]:
def get_sample_rect( patch_h, patch_w, mask_rect, coords, probs):
      
    final_selected_inds = np.random.choice(len(coords), size=[1], p=probs) 
    sample_x = coords[final_selected_inds[0]][1]
    sample_y = coords[final_selected_inds[0]][0]
    sampled_coords = []
    for i in range(patch_h):
        for j in range(patch_w):
            sampled_coords.append([sample_y + i, sample_x + j])
    sampled_coords = np.array(sampled_coords)
    sampled_coords = torch.Tensor(sampled_coords).to(device) # [rect_w*rect_w, 2]
    if (sample_x >= mask_rect[0]) and (sample_y >= mask_rect[1]): 
        ssim_mask = True
    else:
        ssim_mask = False
    if (mask_rect[0] + mask_rect[2] >= sample_x + patch_w) and (mask_rect[1] + mask_rect[3] >= sample_y + patch_h): 
        ssim_mask = ssim_mask and True
    else:
        ssim_mask = False
            
    return sampled_coords, ssim_mask

In [10]:
H, W = 32, 32
coords = np.stack(np.meshgrid(np.linspace(0, H-1, H), np.linspace(0, W-1, W)), -1)  # (H, W, 2)
print(coords)

[[[ 0.  0.]
  [ 1.  0.]
  [ 2.  0.]
  ...
  [29.  0.]
  [30.  0.]
  [31.  0.]]

 [[ 0.  1.]
  [ 1.  1.]
  [ 2.  1.]
  ...
  [29.  1.]
  [30.  1.]
  [31.  1.]]

 [[ 0.  2.]
  [ 1.  2.]
  [ 2.  2.]
  ...
  [29.  2.]
  [30.  2.]
  [31.  2.]]

 ...

 [[ 0. 29.]
  [ 1. 29.]
  [ 2. 29.]
  ...
  [29. 29.]
  [30. 29.]
  [31. 29.]]

 [[ 0. 30.]
  [ 1. 30.]
  [ 2. 30.]
  ...
  [29. 30.]
  [30. 30.]
  [31. 30.]]

 [[ 0. 31.]
  [ 1. 31.]
  [ 2. 31.]
  ...
  [29. 31.]
  [30. 31.]
  [31. 31.]]]


In [12]:
# a = np.array([[1,2,3,4],[5,2,4,5]])
# b = np.array([[1,2,3],[4,5,6],[7,7,8]])

# rays = np.cat([a,b],-1)

a = [[1,2,3,4],[5,2,4,5]]
b = [[1,2,3],[4,5,6],[7,7,8]]

rays = []
for i in range(len(a)):
    rays.append(a[i] + b[i])

print(rays)


[[1, 2, 3, 4, 1, 2, 3], [5, 2, 4, 5, 4, 5, 6]]
